# Model 3: Predict Star Formation Variables (sSFR, SFR, M*, age) Based on Visual Morphology (galaxy image)


In [1]:
#Loading needed modules and classes/functions 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torchvision.datasets import ImageFolder 
from torchvision.io import read_image
from torchvision.io import decode_image
from torchvision.models import vgg19
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import marvin
from marvin.tools.maps import Maps
from marvin.tools.image import Image
from marvin.utils.general.images import get_images_by_list
from marvin import config
from marvin.tools.cube import Cube
from sklearn import metrics
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
import os
import shutil
from PIL import Image as image_PIL

#set config attributes and turn on global downloads of Marvin data
config.setRelease('DR15')
config.mode = 'local'
config.download = True



%matplotlib qt



[INFO]: No release version set. Setting default to DR15
[WARNING]: path /home/juanp/sas/mangawork/manga/spectro/redux/v2_4_3/drpall-v2_4_3.fits cannot be found. Setting drpall to None. (MarvinUserWarning)
[WARNING]: path /home/juanp/sas/mangawork/manga/spectro/analysis/v2_4_3/2.2.1/dapall-v2_4_3-2.2.1.fits cannot be found. Setting dapall to None. (MarvinUserWarning)
[WARNING]: path /home/juanp/sas/mangawork/manga/spectro/redux/v2_4_3/drpall-v2_4_3.fits cannot be found. Setting drpall to None. (MarvinUserWarning)
[WARNING]: path /home/juanp/sas/mangawork/manga/spectro/analysis/v2_4_3/2.2.1/dapall-v2_4_3-2.2.1.fits cannot be found. Setting dapall to None. (MarvinUserWarning)


# Importing Data from Schema Table

In [2]:


data=pd.read_csv('CompleteTable.csv')  #Importing All MaNGA Data from DPRall Schema

galaxy_list=np.loadtxt('Query Results',dtype=str) #Pulling Manga ID's of galaxies which satisfy log(M) > 9 and 0 < z < 0.1


#Problem with image associated with manga id at galaxy_list[3548], mangaid- 1-135668
galaxy_list=np.delete(galaxy_list,3548)

galaxy_list=np.unique(galaxy_list)



galaxy_index=np.zeros(len(galaxy_list)) 
for i in range (len(galaxy_list)): #Getting the index of these galaxies in the schema table
    galaxy_index[i]=np.where(data.loc[:,'mangaid']==galaxy_list[i])[0][0]

galaxy_index=np.array(galaxy_index,dtype=int) #Ensuring we have array that can be used to index, force int 



galaxies=data.iloc[galaxy_index] #DF of galaxies which satisfies the condition, contains all relevant schema data 

galaxies=galaxies.sort_values(by=['plateifu']) #Sorting galaxies by plateifu to match ImageFolder Output 

#Creating the arrays of the independent variables were are interested in, and dependent variable n 

mass=galaxies.loc[:,'nsa_sersic_mass']
log_mass=np.log10(mass)

SFR=galaxies.loc[:,'sfr_tot']
log_SFR=np.log10(SFR)

ha_flux=galaxies.loc[:,'emline_gflux_tot_ha_6564']

mangaid=galaxies.loc[:,'mangaid']

plateifu=galaxies.loc[:,'plateifu']

if plateifu.all()==np.unique(plateifu).all() and mangaid.all()==np.unique(mangaid.all()):
    print('Is Unique List of Galaxies')
    
else:
    print('Try Again')


z=galaxies.loc[:,'z']

n=galaxies.loc[:,'nsa_sersic_n']
n=np.array(n,dtype=np.float32)
n=torch.from_numpy(n).to('cuda:0').reshape(-1,1)

sfr_cat = np.zeros_like(log_SFR)
sfr_cat[log_SFR < -1.5]= int(0)
sfr_cat[log_SFR>-1.5] = int(1)

print(np.mean(sfr_cat))

Is Unique List of Galaxies
0.5341320702145445


# Importing Images from their Downloaded Locations 


In [3]:
# image_locations=[]
# for i in range (len(galaxy_list)):
#     image_locations.append(Image(galaxy_list[i]).filename)
    
# image_locations=np.array(image_locations,dtype=str)
# np.savetxt('Image Directories',image_locations,fmt='%s')

image_locations=np.loadtxt('Image Directories',dtype=str)



In [4]:
# function to resize image
def resize_image(src_image, size=(128,128), bg_color="white"): 
    from PIL import Image, ImageOps 
    
    # resize the image so the longest dimension matches our target size
    src_image.thumbnail(size, Image.ANTIALIAS)
    
    # Create a new square background image
    new_image = Image.new("RGB", size, bg_color)
    
    # Paste the resized image into the center of the square background
    new_image.paste(src_image, (int((size[0] - src_image.size[0]) / 2), int((size[1] - src_image.size[1]) / 2)))
  
    # return the resized image
    return new_image



# Putting the Images into DataLoaders

In [5]:
img_size=(128,128)

# image=ImageFolder('/home/juanp/sas/dr15/manga/spectro/redux/v2_4_3/') #Picks up 3590 pictures, directory list has lenght 3637 however 

# torch.manual_seed(15)
# np.random.seed(15)

image_directory='/home/juanp/sas/dr15/manga/spectro/redux/v2_4_3'
classes= sorted(os.listdir(image_directory))



batch_size=50
image_copies=30

def load_dataset(data_path):
    # Load all the images
    transformation = transforms.Compose([
        # Randomly augment the image data
            # Random horizontal flip
        transforms.RandomHorizontalFlip(0.5),
            # Random vertical flip
        transforms.RandomVerticalFlip(0.3),
        #Rotates the image by some angle
        transforms.RandomRotation(0,360),
        # transform to tensors
        transforms.ToTensor(),
        # Normalize the pixel values (in R, G, and B channels)
        # transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])

    # Load all of the images, transforming them
    full_dataset = torchvision.datasets.ImageFolder(
        root=data_path,
        # transform=transformation
    )
    
    #This loop transforms the images and assigns them the correct label 

    full_dataset_v2 = []   
    for i in range(len(full_dataset)): 
        for j in range(image_copies): #This loops makes it so we have "image_copies" copies of each galaxy image with different orientations 
            temp=transformation(resize_image(full_dataset[i][0]))
            full_dataset_v2.append((temp,sfr_cat[i])) 
    
    full_dataset=full_dataset_v2

    print(len(full_dataset))


    # Split into training (70% and testing (30%) datasets)
    train_size = int(0.7 * len(full_dataset))
    test_size = len(full_dataset) - train_size
    
    # use torch.utils.data.random_split for training/test split
    train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_size])
    
    # define a loader for the training data we can iterate through in 50-image batches
    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=batch_size,
        num_workers=0,
        shuffle=False
    )
    
    # define a loader for the testing data we can iterate through in 50-image batches
    test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=batch_size,
        num_workers=0,
        shuffle=False
    )
        
    return train_loader, test_loader




#####################################################################################################



# Get the iterative dataloaders for test and training data
train_loader, test_loader = load_dataset(image_directory)
batch_size = train_loader.batch_size
print("Data loaders ready to read", image_directory)



107670
Data loaders ready to read /home/juanp/sas/dr15/manga/spectro/redux/v2_4_3


# Defining the Model 

In [6]:
# Create a neural net class
class Net(nn.Module):
    
    
    # Defining the Constructor
    def __init__(self, num_classes=3):
        super(Net, self).__init__()
        
        # In the init function, we define each layer we will use in our model
        
        # Our images are RGB, so we have input channels = 3. 
        # We will apply 12 filters in the first convolutional layer
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=12, kernel_size=3, stride=1, padding=1)
        
        # A second convolutional layer takes 12 input channels, and generates 24 outputs
        self.conv2 = nn.Conv2d(in_channels=12, out_channels=24, kernel_size=3, stride=1, padding=1)
        
        # We in the end apply max pooling with a kernel size of 2
        self.pool = nn.MaxPool2d(kernel_size=2)
        
        # A drop layer deletes 20% of the features to help prevent overfitting
        self.drop = nn.Dropout2d(p=0.2)
        
        # Our 128x128 image tensors will be pooled twice with a kernel size of 2. 128/2/2 is 32.
        # This means that our feature tensors are now 128 x 128, and we've generated 24 of them
        
        # We need to flatten these in order to feed them to a fully-connected layer
        self.fc = nn.Linear(in_features=32 * 32 * 24, out_features=10)

        self.fc1= nn.Linear(in_features=10, out_features=1)


    def forward(self, x):
        
        
        # In the forward function, pass the data through the layers we defined in the init function
        # print(x)
        # Use a ReLU activation function after layer 1 (convolution 1 and pool)
        x=self.conv1(x)
        
        x=self.pool(x)
       

        x=F.relu(x)

        
      
 
     
        # Use a ReLU activation function after layer 2
        x = F.relu(self.pool(self.conv2(x))) 
        
        
        # Select some features to drop to prevent overfitting (only drop during training)
        x = F.dropout(self.drop(x), training=self.training)
        
        # Flatten
        x = x.view(-1, 32 * 32 * 24)
        # Feed to fully-connected layer to predict class
        x = self.fc(x)
        
        # Return class probabilities via a log_softmax function
        x=F.relu(x) 
        
        x= self.fc1(x)
        # print(x)
        return torch.log_softmax(x, dim=1)
    
device = "cpu"
if (torch.cuda.is_available()):
    # if GPU available, use cuda (on a cpu, training will take a considerable length of time!)
    device = "cuda"

# Create an instance of the model class and allocate it to the device
model = Net(num_classes=2).to('cuda')

print(model)

Net(
  (conv1): Conv2d(3, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(12, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (drop): Dropout2d(p=0.2, inplace=False)
  (fc): Linear(in_features=24576, out_features=10, bias=True)
  (fc1): Linear(in_features=10, out_features=1, bias=True)
)


# Creating Training Loop/Function

In [7]:
def train(model, device, train_loader, optimizer, epoch):
    # Set the model to training mode
    model.train()
    train_loss = 0
    print("Epoch:", epoch)
    # Process the images in batches
    for batch_idx, (data, target) in enumerate(train_loader):

        target=target.float()
        # print(target.shape)
        target=target.reshape(-1,1)
        
        # Use the CPU or GPU as appropriate
        # Recall that GPU is optimized for the operations we are dealing with
        data, target = data.to(device), target.to(device)
        
        # Reset the optimizer
        optimizer.zero_grad()
        
        # Push the data forward through the model layers
    
        output = model(data)

        
        # Get the loss
        loss = loss_criteria(output, target)
#         if batch_idx==0:
#             print(output,target,output-target,loss)
#         # Keep a running total
        train_loss += loss.item()
        
        # Backpropagate
        loss.backward()
        optimizer.step()

       
        
        # Print metrics so we see some progress
        # print('\tTraining batch {} Loss: {:.6f}'.format(batch_idx + 1, loss.item()))
            
    # return average loss for the epoch
    avg_loss = train_loss / (batch_idx+1)
    print('Training set: Average loss: {:.6f}'.format(avg_loss))
    return avg_loss

# Create Test Function

In [8]:
def test(model, device, test_loader):
    # Switch the model to evaluation mode (so we don't backpropagate or drop)
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        batch_count = 0
        for data, target in test_loader:
            target=target.float()
            target=target.reshape(-1,1)
            
            batch_count += 1
            data, target = data.to(device), target.to(device)
            
            # Get the predicted classes for this batch
            output = model(data)
            
            # Calculate the loss for this batch
            test_loss += loss_criteria(output, target).item()
            
            # Calculate the accuracy for this batch
            _, predicted = torch.max(output.data, 1)
            correct += torch.sum(target==predicted).item()

    # Calculate the average loss and total accuracy for this epoch
    avg_loss = test_loss / batch_count
    print('Validation set: Average loss: {:.6f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        avg_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
    # return average loss for the epoch
    return avg_loss

# Training the Model

In [9]:
# Use an "Adam" optimizer to adjust weights
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Specify the loss criteria
loss_criteria = nn.BCELoss()

# Track metrics in these arrays
epoch_nums = []
training_loss = []
validation_loss = []




# Train over 10 epochs (We restrict to 10 for time issues)
epochs = 1000
print('Training on', device)
for epoch in range(1, epochs + 1):
        print(epoch)
        train_loss = train(model, device, train_loader, optimizer, epoch)
        test_loss = test(model, device, test_loader)
        epoch_nums.append(epoch)
        training_loss.append(train_loss)
        validation_loss.append(test_loss)
torch.save(model.state_dict(),'/home/juanp/Documents/SURP-2021/Models/Model 6 (SFR Category)/2_Categories')


Training on cuda
1
Epoch: 1
Training set: Average loss: 53.649798
Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

2
Epoch: 2
Training set: Average loss: 53.649798
Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

3
Epoch: 3
Training set: Average loss: 53.649798
Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

4
Epoch: 4
Training set: Average loss: 53.649798
Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

5
Epoch: 5
Training set: Average loss: 53.649798
Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

6
Epoch: 6
Training set: Average loss: 53.649798
Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

7
Epoch: 7
Training set: Average loss: 53.649798
Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

8
Epoch: 8
Training set: Average loss: 53.649798
Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

9
Epoch

Training set: Average loss: 53.649798
Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

68
Epoch: 68
Training set: Average loss: 53.649798
Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

69
Epoch: 69
Training set: Average loss: 53.649798
Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

70
Epoch: 70
Training set: Average loss: 53.649798
Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

71
Epoch: 71
Training set: Average loss: 53.649798
Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

72
Epoch: 72
Training set: Average loss: 53.649798
Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

73
Epoch: 73
Training set: Average loss: 53.649798
Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

74
Epoch: 74
Training set: Average loss: 53.649798
Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

75
Epoch: 75
Training

Training set: Average loss: 53.649798
Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

134
Epoch: 134
Training set: Average loss: 53.649798
Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

135
Epoch: 135
Training set: Average loss: 53.649798
Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

136
Epoch: 136
Training set: Average loss: 53.649798
Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

137
Epoch: 137
Training set: Average loss: 53.649798
Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

138
Epoch: 138
Training set: Average loss: 53.649798
Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

139
Epoch: 139
Training set: Average loss: 53.649798
Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

140
Epoch: 140
Training set: Average loss: 53.649798
Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

141
Epo

Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

199
Epoch: 199
Training set: Average loss: 53.649798
Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

200
Epoch: 200
Training set: Average loss: 53.649798
Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

201
Epoch: 201
Training set: Average loss: 53.649798
Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

202
Epoch: 202
Training set: Average loss: 53.649798
Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

203
Epoch: 203
Training set: Average loss: 53.649798
Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

204
Epoch: 204
Training set: Average loss: 53.649798
Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

205
Epoch: 205
Training set: Average loss: 53.649798
Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

206
Epoch: 206
Training set: Average loss: 53

Training set: Average loss: 53.649798
Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

265
Epoch: 265
Training set: Average loss: 53.649798
Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

266
Epoch: 266
Training set: Average loss: 53.649798
Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

267
Epoch: 267
Training set: Average loss: 53.649798
Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

268
Epoch: 268
Training set: Average loss: 53.649798
Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

269
Epoch: 269
Training set: Average loss: 53.649798
Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

270
Epoch: 270
Training set: Average loss: 53.649798
Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

271
Epoch: 271
Training set: Average loss: 53.649798
Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

272
Epo

...

Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

723
Epoch: 723
Training set: Average loss: 53.649798
Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

724
Epoch: 724
Training set: Average loss: 53.649798
Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

725
Epoch: 725
Training set: Average loss: 53.649798
Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

726
Epoch: 726
Training set: Average loss: 53.649798
Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

727
Epoch: 727
Training set: Average loss: 53.649798
Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

728
Epoch: 728
Training set: Average loss: 53.649798
Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

729
Epoch: 729
Training set: Average loss: 53.649798
Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

730
Epoch: 730
Training set: Average loss: 53

Training set: Average loss: 53.649798
Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

789
Epoch: 789
Training set: Average loss: 53.649798
Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

790
Epoch: 790
Training set: Average loss: 53.649798
Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

791
Epoch: 791
Training set: Average loss: 53.649798
Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

792
Epoch: 792
Training set: Average loss: 53.649798
Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

793
Epoch: 793
Training set: Average loss: 53.649798
Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

794
Epoch: 794
Training set: Average loss: 53.649798
Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

795
Epoch: 795
Training set: Average loss: 53.649798
Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

796
Epo

Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

854
Epoch: 854
Training set: Average loss: 53.649798
Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

855
Epoch: 855
Training set: Average loss: 53.649798
Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

856
Epoch: 856
Training set: Average loss: 53.649798
Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

857
Epoch: 857
Training set: Average loss: 53.649798
Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

858
Epoch: 858
Training set: Average loss: 53.649798
Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

859
Epoch: 859
Training set: Average loss: 53.649798
Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

860
Epoch: 860
Training set: Average loss: 53.649798
Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

861
Epoch: 861
Training set: Average loss: 53

Training set: Average loss: 53.649798
Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

920
Epoch: 920
Training set: Average loss: 53.649798
Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

921
Epoch: 921
Training set: Average loss: 53.649798
Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

922
Epoch: 922
Training set: Average loss: 53.649798
Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

923
Epoch: 923
Training set: Average loss: 53.649798
Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

924
Epoch: 924
Training set: Average loss: 53.649798
Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

925
Epoch: 925
Training set: Average loss: 53.649798
Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

926
Epoch: 926
Training set: Average loss: 53.649798
Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

927
Epo

Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

985
Epoch: 985
Training set: Average loss: 53.649798
Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

986
Epoch: 986
Training set: Average loss: 53.649798
Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

987
Epoch: 987
Training set: Average loss: 53.649798
Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

988
Epoch: 988
Training set: Average loss: 53.649798
Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

989
Epoch: 989
Training set: Average loss: 53.649798
Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

990
Epoch: 990
Training set: Average loss: 53.649798
Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

991
Epoch: 991
Training set: Average loss: 53.649798
Validation set: Average loss: 52.936631, Accuracy: 761250/32301 (2357%)

992
Epoch: 992
Training set: Average loss: 53

In [10]:
plt.figure(figsize=(15,15))
plt.plot(epoch_nums, training_loss)
plt.plot(epoch_nums, validation_loss)
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['training', 'validation'], loc='upper right')
plt.show()

In [18]:


# Defining Labels and Predictions
import seaborn as sns
classes=[0,1]
truelabels = []
predictions = []
model.eval()
print("Getting predictions from test set...")
for data, target in test_loader:
    data, target= data.to('cuda'), target.to('cuda')
    for label in target.data.cpu().numpy():
        truelabels.append(label)
    for prediction in model(data).data.cpu().numpy().argmax(1):
        predictions.append(prediction) 

# Plot the confusion matrix
cm = confusion_matrix(truelabels, predictions)
tick_marks = np.arange(len(classes))

df_cm = pd.DataFrame(cm, index = classes, columns = classes)
plt.figure(figsize = (7,7))
sns.heatmap(df_cm, annot=True, cmap=plt.cm.Blues, fmt='g')
plt.xlabel("Predicted Shape", fontsize = 20)
plt.ylabel("True Shape", fontsize = 20)
plt.show()



Getting predictions from test set...
